In [1]:
lsb_release -aimport nx

graph = { 'source':{'f':(303934,455,0)},
                    #    Capacidad max, coste, cuanto se movio si es mayor que cero es importante
    'f': {'A': (5, 1, 0), 'D': (2, 2, 0)},
    'A': {'C': (12, 3, 0),'sink':(4,7,0)},
    'C': { 'D': (20, 7, 0)},
    'D': {'C': (7, 8, 0), 'sink': (6, 9, 0)},
    'sink': {}
}


In [2]:
def min_cost_max_flow(graph, source, sink):
    max_flow = 0
    parent = {}

    def bfs():
        visited = set()
        queue = [(source, float("Inf"))]

        while queue:
            node, path_flow = queue.pop(0)
            visited.add(node)

            for neighbor, (capacity, cost, flow) in graph[node].items():
                if neighbor not in visited and capacity > flow:
                    parent[neighbor] = (node, min(path_flow, capacity - flow))
                    if neighbor == sink:
                        return parent[neighbor][1]
                    queue.append((neighbor, parent[neighbor][1]))

        return 0

    while True:
        path_flow = bfs()
        if path_flow == 0:
            break

        max_flow += path_flow
        v = sink
        while v != source:
            u, flow = parent[v]
            new_flow = min(graph[u][v][0], graph[u][v][2] + path_flow)
            if new_flow <= graph[u][v][0]:
                graph[u][v] = (graph[u][v][0], graph[u][v][1], new_flow)
            if v in graph and u in graph[v]:
                graph[v][u] = (graph[v][u][0], graph[v][u][1], graph[v][u][2] - path_flow)
            else:
                graph[v][u] = (0, -graph[u][v][1], -path_flow)
            v = u

    return max_flow, graph
source = 'source'
sink = 'sink'

max_flow,graph = min_cost_max_flow(graph, source, sink)
print(f"Flujo máximo con costo mínimo: {max_flow}")

Flujo máximo con costo mínimo: 7


# Para crear el grafo

In [3]:
def create_graph(sink_name:str,count_want:int,ware_houses_list:list[tuple[str,int,float]],manufacturer_list:list[tuple]):
    """
    Cada lista recibe tuplas con 
    nombre de la empresa o su id
    cant maxima abastecer y precio por  unidad sumado con el del transportista
    :param sink_name: 
    :param count_want: 
    :param ware_houses_list: 
    :param manufacturer_list: 
    :return: 
    """

    fake_sink='sink'
    source='source'
    graph={sink_name:{},fake_sink:{sink_name:(count_want,0,0)}}
    graph[source]={}

    for tupple in ware_houses_list:
        name,count_supply,price=tupple
        a=(count_supply,price,0)
        graph[name]={fake_sink:a}
        dic=graph[source]

        dic[name]=(count_want,0,0)
    return graph





create_graph('Tienda_1',8,[('Proveedor_1',80,80.8),('Proveedor_2',40,25.5)],[])




{'Tienda_1': {},
 'sink': {'Tienda_1': (8, 0, 0)},
 'source': {'Proveedor_1': (8, 0, 0), 'Proveedor_2': (8, 0, 0)},
 'Proveedor_1': {'sink': (80, 80.8, 0)},
 'Proveedor_2': {'sink': (40, 25.5, 0)}}

In [4]:
graph=create_graph('Tienda_1',9,[('Proveedor_1',80,80.8),('Proveedor_2',4,4)],[])

max_flow,graph = min_cost_max_flow(graph, source,'Tienda_1')
print(f"Flujo máximo con costo mínimo: {max_flow}")

Flujo máximo con costo mínimo: 9


In [5]:
print(graph)

{'Tienda_1': {'sink': (0, 0, -9)}, 'sink': {'Tienda_1': (9, 0, 9), 'Proveedor_2': (0, -4, -9)}, 'source': {'Proveedor_1': (9, 0, 0), 'Proveedor_2': (9, 0, 9)}, 'Proveedor_1': {'sink': (80, 80.8, 0)}, 'Proveedor_2': {'sink': (4, 4, 4), 'source': (0, 0, -9)}}


Notar que hay un receptor falso para que envie a la tienda osea el proposito es que no lleguen mas productos de lo que se quiere solo tomar los numeros de extrema derecha de la tupla positivos el resto es de como esta el caudal 

In [6]:
import heapq

def min_cost_max_flow(graph, source, sink):
    max_flow = 0
    min_cost = 0
    parent = {}
    dist = {}

    def bellman_ford():
        for node in graph:
            dist[node] = float('inf')
        dist[source] = 0
        for _ in range(len(graph) - 1):
            for node in graph:
                for neighbor, (capacity, cost, flow) in graph[node].items():
                    if capacity > flow and dist[node] + cost < dist[neighbor]:
                        dist[neighbor] = dist[node] + cost
                        parent[neighbor] = node

    def dijkstra():
        heap = [(0, source)]
        while heap:
            d, node = heapq.heappop(heap)
            if d != dist[node]:
                continue
            for neighbor, (capacity, cost, flow) in graph[node].items():
                if capacity > flow and dist[node] + cost < dist[neighbor]:
                    dist[neighbor] = dist[node] + cost
                    parent[neighbor] = node
                    heapq.heappush(heap, (dist[neighbor], neighbor))

    while True:
        bellman_ford()
        if dist[sink] == float('inf'):
            break
        dijkstra()
        if dist[sink] == float('inf'):
            break

        path_flow = float('inf')
        v = sink
        while v != source:
            u = parent[v]
            path_flow = min(path_flow, graph[u][v][0] - graph[u][v][2])
            v = u

        max_flow += path_flow
        min_cost += path_flow * dist[sink]
        v = sink
        while v != source:
            u = parent[v]
            graph[u][v] = (graph[u][v][0], graph[u][v][1], graph[u][v][2] + path_flow)
            if v in graph and u in graph[v]:
                graph[v][u] = (graph[v][u][0], graph[v][u][1], graph[v][u][2] - path_flow)
            else:
                graph[v][u] = (0, -graph[u][v][1], -path_flow)
            v = u

    return max_flow, min_cost, graph

In [7]:
graph=create_graph('Tienda_1',9,[('Proveedor_1',80,80.8),('Proveedor_2',4,4)],[])

max_flow,graph = min_cost_max_flow(graph, source,'Tienda_1')
print(f"Flujo máximo con costo mínimo: {max_flow}")

ValueError: too many values to unpack (expected 2)

In [ ]:
print(graph)

In [12]:
import networkx as nx
G = nx.DiGraph()
fuente='Fuente'
prove_1='Provedor_1'
prove_2='Provedor_2'
fake_sink='Fake_sink'
Tienda_1='Tienda_1'
G.add_edges_from(
    [
        (fuente,prove_1, {"capacity":9, "weight": 0}),
        (fuente,prove_2, {"capacity": 9, "weight": 0}),
        (prove_1,fake_sink, {"capacity": 80, "weight": 80}),
        (prove_2,fake_sink,{"capacity": 4, "weight": 10}),
        (fake_sink,
         Tienda_1,{"capacity": 9,"weight": 9}),
        
    ]
)
mincostFlow = nx.max_flow_min_cost(G, fuente, Tienda_1)
mincost = nx.cost_of_flow(G, mincostFlow)
mincost
373
from networkx.algorithms.flow import maximum_flow
#maxFlow = maximum_flow(G, 1, 7)[1]
#nx.cost_of_flow(G, maxFlow) >= mincost
#True
#mincostFlowValue = sum((mincostFlow[u][7] for u in G.predecessors(7))) - sum(
#    (mincostFlow[7][v] for v in G.successors(7))
#)
#mincostFlowValue == nx.maximum_flow_value(G, 1, 7)
#True

KeyboardInterrupt: 

In [10]:
# Python3 program to implement
# the above approach
from sys import maxsize
from typing import List

# Stores the found edges
found = []

# Stores the number of nodes
N = 0

# Stores the capacity
# of each edge
cap = []

flow = []

# Stores the cost per
# unit flow of each edge
cost = []

# Stores the distance from each node
# and picked edges for each node
dad = []
dist = []
pi = []

INF = maxsize // 2 - 1

# Function to check if it is possible to
# have a flow from the src to sink
def search(src: int, sink: int) -> bool:

	# Initialise found[] to false
	found = [False for _ in range(N)]

	# Initialise the dist[] to INF
	dist = [INF for _ in range(N + 1)]

	# Distance from the source node
	dist[src] = 0

	# Iterate until src reaches N
	while (src != N):
		best = N
		found[src] = True

		for k in range(N):

			# If already found
			if (found[k]):
				continue

			# Evaluate while flow
			# is still in supply
			if (flow[k][src] != 0):

				# Obtain the total value
				val = (dist[src] + pi[src] -
						pi[k] - cost[k][src])

				# If dist[k] is > minimum value
				if (dist[k] > val):

					# Update
					dist[k] = val
					dad[k] = src

			if (flow[src][k] < cap[src][k]):
				val = (dist[src] + pi[src] -
						pi[k] + cost[src][k])

				# If dist[k] is > minimum value
				if (dist[k] > val):

					# Update
					dist[k] = val
					dad[k] = src

			if (dist[k] < dist[best]):
				best = k

		# Update src to best for
		# next iteration
		src = best

	for k in range(N):
		pi[k] = min(pi[k] + dist[k], INF)

	# Return the value obtained at sink
	return found[sink]

# Function to obtain the maximum Flow
def getMaxFlow(capi: List[List[int]], 
			costi: List[List[int]], 
			src: int, sink: int) -> List[int]:

	global cap, cost, found, dist, pi, N, flow, dad
	cap = capi
	cost = costi

	N = len(capi)
	found = [False for _ in range(N)]
	flow = [[0 for _ in range(N)]
			for _ in range(N)]
	dist = [INF for _ in range(N + 1)]
	dad = [0 for _ in range(N)]
	pi = [0 for _ in range(N)]

	totflow = 0
	totcost = 0

	# If a path exist from src to sink
	while (search(src, sink)):

		# Set the default amount
		amt = INF
		x = sink
		
		while x != src:
			amt = min(
				amt, flow[x][dad[x]] if
				(flow[x][dad[x]] != 0) else
				cap[dad[x]][x] - flow[dad[x]][x])
			x = dad[x]

		x = sink
		
		while x != src:
			if (flow[x][dad[x]] != 0):
				flow[x][dad[x]] -= amt
				totcost -= amt * cost[x][dad[x]]

			else:
				flow[dad[x]][x] += amt
				totcost += amt * cost[dad[x]][x]
				
			x = dad[x]

		totflow += amt

	# Return pair total cost and sink
	return [totflow, totcost]

# Driver Code
if __name__ == "__main__":

	s = 0
	t = 4

	cap = [ [ 0, 3, 1, 0, 3 ], 
			[ 0, 0, 2, 0, 0 ], 
			[ 0, 0, 0, 1, 6 ], 
			[ 0, 0, 0, 0, 2 ],
			[ 0, 0, 0, 0, 0 ] ]

	cost = [ [ 0, 1, 0, 0, 2 ], 
			[ 0, 0, 0, 3, 0 ], 
			[ 0, 0, 0, 0, 0 ], 
			[ 0, 0, 0, 0, 1 ],
			[ 0, 0, 0, 0, 0 ] ]

	ret = getMaxFlow(cap, cost, s, t)

	print("{} {}".format(ret[0], ret[1]))

# This code is contributed by sanjeev2552


6 8


In [11]:
import heapq

In [ ]:
def min_cost_max_flow(graph, source, sink):
    max_flow = 0
    min_cost = 0
    parent = {}
    dist = {}
    paths = []  # Lista para guardar los caminos

    def bellman_ford():
        for node in graph:
            dist[node] = float('inf')
        dist[source] = 0
        for _ in range(len(graph) - 1):
            for node in graph:
                for neighbor, (capacity, cost, flow) in graph[node].items():
                    if capacity > flow and dist[node] + cost < dist[neighbor]:
                        dist[neighbor] = dist[node] + cost
                        parent[neighbor] = node

    def dijkstra():
        heap = [(0, source)]
        while heap:
            d, node = heapq.heappop(heap)
            if d != dist[node]:
                continue
            for neighbor, (capacity, cost, flow) in graph[node].items():
                if capacity > flow and dist[node] + cost < dist[neighbor]:
                    dist[neighbor] = dist[node] + cost
                    parent[neighbor] = node
                    heapq.heappush(heap, (dist[neighbor], neighbor))

    while True:
        bellman_ford()
        if dist[sink] == float('inf'):
            break
        dijkstra()
        if dist[sink] == float('inf'):
            break

        path_flow = float('inf')
        v = sink
        while v != source:
            u = parent[v]
            path_flow = min(path_flow, graph[u][v][0] - graph[u][v][2])
            v = u

        max_flow += path_flow
        min_cost += path_flow * dist[sink]
        v = sink
        while v != source:
            u = parent[v]
            graph[u][v] = (graph[u][v][0], graph[u][v][1], graph[u][v][2] + path_flow)
            if v in graph and u in graph[v]:
                graph[v][u] = (graph[v][u][0], graph[v][u][1], graph[v][u][2] - path_flow)
            else:
                graph[v][u] = (0, -graph[u][v][1], -path_flow)
            v = u

        paths.append(parent.copy())  # Guardar el camino

    return max_flow, min_cost, graph, paths  # Devolver los caminos junto con los otros resultados

In [ ]:
graph=create_graph('Tienda_1',9,[('Proveedor_1',80,80.8),('Proveedor_2',4,4)],[])

max_flow, min_cost, graph, paths = min_cost_max_flow(graph, source,'Tienda_1')
print(f"Flujo máximo con costo mínimo: {max_flow}")

In [2]:
lis=[4]

lis.extend([5])

print(lis)

[4, 5]
